In [1]:
import requests
from langchain.document_loaders import TextLoader

In [2]:
with open("刑法.txt", "r") as f:
    data = f.readlines()
data = [d.strip() for d in data]
data[:2], len(data)

(['第一条 为了惩罚犯罪，保护人民，根据宪法，结合我国同犯罪作斗争的具体经验及实际情况，制定本法。',
  '第二条 中华人民共和国刑法的任务，是用刑罚同一切犯罪行为作斗争，以保卫国家安全，保卫人民民主专政的政权和社会主义制度，保护国有财产和劳动群众集体所有的财产，保护公民私人所有的财产，保护公民的人身权利、民主权利和其他权利，维护社会秩序、经济秩序，保障社会主义建设事业的顺利进行。'],
 504)

In [4]:
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceBgeEmbeddings
import torch

model_path = "/mnt/nfs/zsd_server/models/huggingface/embedding_models/BAAI/bge-large-zh-v1.5/"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
embedding_model.query_instruction = "为这个句子生成表示以用于检索相关文章："

embedding_model

/home/zsd/miniconda3/envs/huggingface/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='/mnt/nfs/zsd_server/models/huggingface/embedding_models/BAAI/bge-large-zh-v1.5/', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='为这个句子生成表示以用于检索相关文章：', embed_instruction='')

In [5]:
data_embeddings = embedding_model.embed_documents(data)
len(data_embeddings), len(data_embeddings[0])

(504, 1024)

In [6]:
text_embedding_pairs = zip(data, data_embeddings)
text_embedding_pairs

In [7]:
import pickle

pickle.dump(text_embedding_pairs, open('text_embedding_pairs_BAAI.pkl', 'wb'))

In [8]:
from langchain_community.vectorstores import FAISS

# vectorstore = FAISS.from_documents(documents=data, embedding=embedding_model)
# vectorstore
# vectorstore.save_local(folder_path="vectorstore")

In [9]:
faiss = FAISS.from_embeddings(text_embedding_pairs, embedding_model)
faiss

In [10]:
retriever = faiss.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7aa65a471e90>)

In [11]:
from langchain.prompts import ChatPromptTemplate

template = """你是问答任务助手。使用以下检索到的上下文片段来回答问题。如果你不知道答案，就说你不知道。最多使用三个句子，保持答案简洁。
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你是问答任务助手。使用以下检索到的上下文片段来回答问题。如果你不知道答案，就说你不知道。最多使用三个句子，保持答案简洁。\nQuestion: {question} \nContext: {context} \nAnswer:\n'))]


In [13]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline

base_model_path = "/mnt/nfs/zsd_server/models/huggingface/chinese-alpaca-2-7b/"

base_model = LlamaForCausalLM.from_pretrained(
    base_model_path,
    # load_in_8bit=True,
    device_map='auto',
    torch_dtype=torch.bfloat16,
)
base_model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(55296, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

In [15]:
from langchain.llms import HuggingFacePipeline

tokenizer = LlamaTokenizer.from_pretrained(base_model_path)

pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=4096,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample=True,
)

local_llm = HuggingFacePipeline(pipeline=pipe)
local_llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7aa5b00b5d50>)

In [21]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | local_llm
    | StrOutputParser() 
)

rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7aa65a471e90>),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你是问答任务助手。使用以下检索到的上下文片段来回答问题。如果你不知道答案，就说你不知道。最多使用三个句子，保持答案简洁。\nQuestion: {question} \nContext: {context} \nAnswer:\n'))])
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7aa5b00b5d50>)
| StrOutputParser()

In [23]:
query = "持有管制刀具怎么判？"
rag_chain.invoke(query)

"Human: 你是问答任务助手。使用以下检索到的上下文片段来回答问题。如果你不知道答案，就说你不知道。最多使用三个句子，保持答案简洁。\nQuestion: 持有管制刀具怎么判？ \nContext: [Document(page_content='第二百九十七条 违反法律规定，携带武器、管制刀具或者爆炸物参加集会、游行、示威的，处三年以下有期徒刑、拘役、管制或者剥夺政治权利。'), Document(page_content='第一百三十条 非法携带枪支、弹药、管制刀具或者爆炸性、易燃性、放射性、毒害性、腐蚀性物品，进入公共场所或者公共交通工具，危及公共安全，情节严重的，处三年以下有期徒刑、拘役或者管制。'), Document(page_content='第一百二十八条 违反枪支管理规定，非法持有、私藏枪支、弹药的，处三年以下有期徒刑、拘役或者管制；情节严重的，处三年以上七年以下有期徒刑。依法配备公务用枪的人员，非法出租、出借枪支的，依照前款的规定处罚。依法配置枪支的人员，非法出租、出借枪支，造成严重后果的，依照第一款的规定处罚。单位犯第二款、第三款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照第一款的规定处罚。'), Document(page_content='第一百二十条之六 明知是宣扬恐怖主义、极端主义的图书、音频视频资料或者其他物品而非法持有，情节严重的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金。')] \nAnswer:\n根据中国刑法相关规定,持械参与集会等行为属于违法行为.如果情节较轻会被罚款或拘留,如情节较为恶劣则可能被判处3年以下有期徒刑、拘役或者管制;而非法持有枪支和炸弹的行为将被视为犯罪并且将会受到更严厉的惩罚.(参见第102条)"

In [24]:
local_llm(query)

'持有管制刀具怎么判？第159条：禁止携带、运输危险物品，违者处五年以下有期徒刑或者拘役；情节严重的，处以五年以上十年以下有期徒刑。这是指在公共场所非法使用或私藏枪支弹药等具有杀伤力的工具的行为属于刑法规定中的危害社会罪行为之一——暴力犯罪（第三百零七条）和聚众斗殴罪行（第二百九十条第一款）两个罪名中任意一个构成要件的规定情况下的情形下才可能被认定为"持枪抢劫案".而如果仅仅是因携带了一把匕首或是其它类似性质的小型武器就单独定性于“持刀伤害”一类的刑事案件当中.\n总之，根据我国现行法律制度以及司法解释实施细则所确立的标准来看的话，只要当事人依法依规地进行了合法登记并遵守相关法规要求的情况下，他/她就不会因为自己拥有一支步枪就被判定犯下了“持枪抢劫案”这样的重罪犯事件'

In [25]:
sim_docs = faiss.similarity_search(query)
sim_docs

[Document(page_content='第二百九十七条 违反法律规定，携带武器、管制刀具或者爆炸物参加集会、游行、示威的，处三年以下有期徒刑、拘役、管制或者剥夺政治权利。'),
 Document(page_content='第一百三十条 非法携带枪支、弹药、管制刀具或者爆炸性、易燃性、放射性、毒害性、腐蚀性物品，进入公共场所或者公共交通工具，危及公共安全，情节严重的，处三年以下有期徒刑、拘役或者管制。'),
 Document(page_content='第一百二十八条 违反枪支管理规定，非法持有、私藏枪支、弹药的，处三年以下有期徒刑、拘役或者管制；情节严重的，处三年以上七年以下有期徒刑。依法配备公务用枪的人员，非法出租、出借枪支的，依照前款的规定处罚。依法配置枪支的人员，非法出租、出借枪支，造成严重后果的，依照第一款的规定处罚。单位犯第二款、第三款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照第一款的规定处罚。'),
 Document(page_content='第一百二十条之六 明知是宣扬恐怖主义、极端主义的图书、音频视频资料或者其他物品而非法持有，情节严重的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金。')]

In [26]:
local_llm(f"Human: 你是问答任务助手。使用以下检索到的上下文片段来回答问题。如果你不知道答案，就说你不知道。最多使用三个句子，保持答案简洁。\nQuestion: {query} \nContext: {','.join([d.page_content for d in sim_docs])}\nAnswer:\n")

'Human: 你是问答任务助手。使用以下检索到的上下文片段来回答问题。如果你不知道答案，就说你不知道。最多使用三个句子，保持答案简洁。\nQuestion: 持有管制刀具怎么判？ \nContext: 第二百九十七条 违反法律规定，携带武器、管制刀具或者爆炸物参加集会、游行、示威的，处三年以下有期徒刑、拘役、管制或者剥夺政治权利。,第一百三十条 非法携带枪支、弹药、管制刀具或者爆炸性、易燃性、放射性、毒害性、腐蚀性物品，进入公共场所或者公共交通工具，危及公共安全，情节严重的，处三年以下有期徒刑、拘役或者管制。,第一百二十八条 违反枪支管理规定，非法持有、私藏枪支、弹药的，处三年以下有期徒刑、拘役或者管制；情节严重的，处三年以上七年以下有期徒刑。依法配备公务用枪的人员，非法出租、出借枪支的，依照前款的规定处罚。依法配置枪支的人员，非法出租、出借枪支，造成严重后果的，依照第一款的规定处罚。单位犯第二款、第三款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照第一款的规定处罚。,第一百二十条之六 明知是宣扬恐怖主义、极端主义的图书、音频视频资料或者其他物品而非法持有，情节严重的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金。\nAnswer:\n根据《中华人民共和国刑法》相关规定：1.持械参与集会有关法规第297条规定了相关判罚标准（即3年以下有期徒刑或拘留），如果属于"公然抗拒执法警力""斗殴致人重伤致死等情况"则可加大罚款数额甚至判刑长度。2.未按规定携带枪支而入非禁止区域内涉及上述犯罪行为之一的情况将被认定为违法，可能会面临相应的行政罚款或其他惩罚措施'